# PROCEDIMIENTO DE PARA CORREGIR PREFIJO OBEJETOS TRAS RESTAURACION

Este cuaderno arregla los nombres de bases de la base de datos de todos los objetos tras un cambio de nombre en base de datos. Esta situacion se suele dar cuando se restaura una base de datos. 

## INTRUCCIONES DE USO

Es necesario definir los parámetros según se quieran cambiar los nombres de los prefijos. Hay que poner en la variable "BuscarBaseDeDatos" el prefijo que se va a buscar y cambiar por la nueva base de datos, que hay que escribir en la variable "ReemplazarBaseDeDatos". Estas variables se definen en PARAMETROS.

Tras haber hecho este paso previo se ejecuta el código hasta la sección de  PROCESO DE REEMPLAZAMIENTO. Aquí de nuevo se ejecuta el bloque, se introduce la instancia en donde se ha hecho la restauración y automáticamente se renombrarán aquellos objetos con los prefijos antiguos.

Si se ejecuta el programa desde un ordenador local, es necesaria tener activada la VPN. 

## INSTALACION DE PAQUETES

In [ ]:
!pip install prettytable
import pyodbc
import pandas as pd
from prettytable import PrettyTable
import re

## DECLARACION DE PARAMETROS Y FUNCIONES

### PARAMETROS 

In [ ]:
BuscarBaseDeDatos = '...'
ReemplazarBaseDeDatos = '...'
BuscarCreate = 'Create'
ReemplazarCreate  = 'ALTER'

### FUNCIONES

#### Obtencion y modificacion de codigo

In [ ]:
def ObtenerCodigoActual(nombre_procedimiento, esquema):
    """
    Recupera el código fuente (definition) de un procedimiento almacenado (Stored Procedure)
    desde SQL Server, a partir del nombre del procedimiento y su esquema.

    Parámetros:
    -----------
    nombre_procedimiento : str
        Nombre del procedimiento almacenado.
    esquema : str
        Nombre del esquema al que pertenece el procedimiento (por ejemplo, 'dbo' o 'dba').

    Retorna:
    --------
    str
        El código SQL del procedimiento almacenado si se encuentra.
    
    int
        Devuelve 1 si no se encuentra el procedimiento (como señal de error).

    Notas:
    ------
    - Utiliza una consulta con parámetros para evitar inyección SQL.
    - El procedimiento debe tener su definición disponible (no encriptado).
    - Utiliza pandas para ejecutar y obtener la consulta como DataFrame.
    - Cierra la conexión al finalizar, independientemente del resultado.

    Ejemplo de uso:
    ---------------
    >>> ObtenerCodigoActual("MySp", "dbo")
    'CREATE PROCEDURE dbo.MySp AS BEGIN ... END'
    """

    # Crear conexión
    conn = pyodbc.connect(conn_str)
    
    queryprocedimiento = """
        SELECT sm.definition
        FROM sys.sql_modules sm
        JOIN sys.objects o ON sm.object_id = o.object_id
        JOIN sys.schemas s ON o.schema_id = s.schema_id
        WHERE o.name = ? AND s.name = ?
    """

    codigoProcedimiento = pd.read_sql(queryprocedimiento, conn, params=[nombre_procedimiento, esquema])

    if codigoProcedimiento.empty:
        print(f"No se encontró el procedimiento {esquema}.{nombre_procedimiento}")
        return 1

    # Cerrar conexión
    conn.close()

    # Extraer el código SQL del procedimiento desde el DataFrame
    codigoProcedimiento = codigoProcedimiento.iloc[0, 0]
    
    return codigoProcedimiento


In [ ]:
def ModificarCodigo(codigoModificado,procedimiento):
    error=0
    
    try:
        # Establecer conexión
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        # Ejecutar la consulta
        cursor.execute(codigoModificado)
        
    except pyodbc.Error as e:
        print(f"Error al ejecutar SQL en procedimiento {procedimiento}:")
        print(e)
        
        error=1
        conn.rollback()

        print(f"El código que produce error es: \n {codigoModificado}")
        
        return 1
        
    finally:
        if error == 0:
            conn.commit()
            
        cursor.close()
        conn.close()
        return error
        

#### Elegir conexion

In [ ]:
def ElegirConexion():
    """
    Solicita al usuario que seleccione una instancia de base de datos y configura los parámetros
    de conexión globales necesarios para conectarse a un servidor SQL Server.

    La función realiza lo siguiente:
    1. Solicita al usuario introducir un número del 1 al 8, correspondiente a una instancia predefinida.
    2. Utiliza el número ingresado para seleccionar el nombre de la instancia desde la lista 'instancias'.
    3. Obtiene la información de conexión (servidor y contraseña) desde el diccionario 'conexionesDDBB'.
    4. Asigna valores a las variables globales:
       - `server`: nombre del servidor SQL
       - `password`: contraseña para la conexión
       - `conn_str`: cadena de conexión completa ODBC para SQL Server

    Variables globales modificadas:
        - server (str): nombre del servidor SQL seleccionado.
        - password (str): contraseña asociada al usuario SQL.
        - conn_str (str): cadena de conexión ODBC construida con los valores seleccionados.

    Dependencias:
        - 'instancias': lista indexada de nombres de instancia disponibles.
        - 'conexionesDDBB': diccionario con credenciales y servidores por instancia.
        - 'BaseDeDatos': nombre de la base de datos a la que se conecta.
        - 'login': nombre de usuario SQL.

    Ejemplo de uso:
        >>> ElegirConexion()
        (el usuario ingresa "1")
        (se configuran las variables globales para la instancia BI-PRO)

    Notas:
        - La función no valida si el número ingresado está dentro del rango 1–8.
        - Podría lanzar un IndexError si el número ingresado es incorrecto.
        - Considera agregar validación para mejorar la robustez.
    """

    instancia=int(input('Introduce el numero que corresponda: 1.Instancia1 2.Instancia2 3.Instancia3 4.Instancia4 5.Instancia15 6.Instancia16 7.Instancia17 8.Instancia8 \n'))
    if instancia>8:
        print("Error, el numero no es valido")
        return 1
    nombre_instancia=instancias[instancia-1]

    
    
    global server
    server = conexionesDDBB[nombre_instancia][0]
    
    global password
    password=conexionesDDBB[nombre_instancia][1]
    
    global conn_str
    conn_str = (
    'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={BaseDeDatos};'
    f'UID={login};'
    f'PWD={password}'
    )
    return 0

#### Obtencion de listas

In [ ]:
def GeneradorLista():
    error=0
    lista=[]
    
    try:
        # Establecer conexión
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        buscar = "%" + ReemplazarBaseDeDatos + "%" 
        
        # Ejecutar la consulta
        consulta="""SELECT	sm.definition,
                            o.name,
                            s.name,
                    		CASE 
                    			WHEN definition like '%PROCEDURE%'  THEN 'procedimiento'
                    			WHEN definition like '%PROC%'  THEN 'procedimiento'
                    			WHEN definition like '%VIEW%'		THEN 'vista' 
                    			WHEN definition like '%TRIGGER%'	THEN 'trigger'
                    			WHEN definition like '%FUNCTION%'	THEN 'funcion'
                    			ELSE 'otro' END AS tipo
                    FROM sys.sql_modules sm
                    JOIN sys.objects o 
                        ON sm.object_id = o.object_id
                    JOIN sys.schemas s 
                        ON o.schema_id=s.schema_id
                    WHERE definition like ?"""
        cursor.execute(consulta,buscar)

        
        records = cursor.fetchall()
        for r in records:
            codigo = r[0]
            nombre_objeto = r[1]
            esquema = r[2]
            tipo=r[3]            

            lista.append({   "Tipo" : tipo,
                             "Esquema": esquema,
                             "Nombre" : nombre_objeto,
                             "Codigo" : codigo})
            
    except pyodbc.Error as e:
        print(f"Error al ejecutar SQL la consulta:")
        print(e)
        print(f"La consulta realizada ha sido {consulta}")

        error=1
        conn.rollback()
        
        return 1, None
        
    finally:
        if error == 0:
            conn.commit()
            
        cursor.close()
        conn.close()
        
        return error,lista

In [ ]:
def GeneradorListaJobs():
    error=0
    lista=[]
    
    try:
        # Establecer conexión
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        
        buscar = "%" + ReemplazarBaseDeDatos + "%" 
        
        # Ejecutar la consulta
        cursor.execute("""SELECT A.TipoDeObjeto,
                    			A.NombreDeObjeto,
                    			A.Contenido
                    	FROM(SELECT 
                    			'job' AS TipoDeObjeto,
                    			B.NombreJob AS NombreDeObjeto,
                    			STUFF((SELECT ','+'P'+CAST(js.step_id AS VARCHAR(10))+':'+js.step_name+' - COMMAND: '+js.command
                    																FROM msdb.dbo.sysjobsteps js
                    																JOIN msdb.dbo.sysjobs sj 
                    																	ON sj.job_id=js.job_id
                    																WHERE js.job_id = sj.job_id
                    																and B.NombreJob=sj.name COLLATE SQL_Latin1_General_CP1_CI_AS
                    																ORDER BY js.step_id
                    																FOR XML PATH(''), TYPE).value('.', 'NVARCHAR(MAX)'), 1, 2, '') AS Contenido
                    		FROM(	SELECT j.name AS NombreJob
                    				From msdb.dbo.sysjobs j) AS B ) AS A
                            WHERE A.Contenido like ? """,[buscar])

        records = cursor.fetchall()
        for r in records:
            codigo = r[2]
            nombre_objeto = r[1]
            tipo=r[0]            
           
            lista.append({   "Tipo" : tipo,
                             "Nombre" : nombre_objeto,
                             "Codigo" : codigo})
            
    except pyodbc.Error as e:    
        if not records:
            print("No hay ningún paso en ningún job que tenga dependencias.")
        else:
            print(f"Error al ejecutar SQL la consulta:")
            print(e)
            print(f"La consulta realizada ha sido {consulta}")

        error=1
        conn.rollback()
        
        return 1, None
        
    finally:
        if error == 0:
            conn.commit()
            
        cursor.close()
        conn.close()
        
        return error,lista

## DATOS DE CONEXION

In [ ]:
conexionesDDBB={ "Instancia1" : ["direccion_Instancia1","password_Instancia1"], ...}
instancias=["Instancia1","Instancia2" ,....]
login = 'login'
BaseDeDatos = ReemplazarBaseDeDatos

## PROCESO DE REEMPLAZAMIENTO

In [ ]:
Error=ElegirConexion()
if Error==0:
    errorlista,lista=GeneradorLista()
    
    if errorlista == 0:
        for item in lista:
            nombre=item['Nombre']
            esquema=item['Esquema']
            tipo=item['Tipo']
            codigoOriginal=item['Codigo']
    
            if codigoOriginal != 1:    
                codigoModificado = re.sub(BuscarCreate,ReemplazarCreate, codigoOriginal, flags=re.IGNORECASE)
                codigoModificado = re.sub(BuscarBaseDeDatos,ReemplazarBaseDeDatos, codigoModificado, flags=re.IGNORECASE)
            
                #Ejecutar en base de datos el procedimiento corregido 
                flag=ModificarCodigo(codigoModificado,nombre)
                if flag == 0:
                    print(f"El objeto {esquema}.{nombre} de tipo {tipo}  ha sido modificado con éxito")
                    
    errorlistajobs,listajobs=GeneradorListaJobs()
    if errorlistajobs == 0:
        print("Se procede a listar los jobs que necesitan cambio manual en alguno de sus pasos:")
        for item in listajobs:
            nombre=item['Nombre']
            print(f"{nombre}")
else:
    print("Ninguna modificacion realizada")